In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 62.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=88acf653333c6f7f95ff784f32249233cc3262e323d62e2df2bac4c2a9c97b2b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test").config("spark.some.config.option", "some-value").getOrCreate()

In [4]:
spark

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path=("/content/drive/MyDrive/Colab Notebooks/ParisHousingClass.csv")

In [7]:
df=spark.read.csv(path, inferSchema= True, header=True)

df.show()

df.printSchema()

df.na.drop().show()


+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+
|squareMeters|numberOfRooms|hasYard|hasPool|floors|cityCode|cityPartRange|numPrevOwners|made|isNewBuilt|hasStormProtector|basement|attic|garage|hasStorageRoom|hasGuestRoom|    price|category|
+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+
|       75523|            3|      0|      1|    63|    9373|            3|            8|2005|         0|                1|    4313| 9005|   956|             0|           7|7559081.5|   Basic|
|       80771|           39|      1|      1|    98|   39381|            8|            6|2015|         1|                0|    3653| 2436|   128|             1|           2|8085989.5|  Luxury|
|       55712|           58|      0|    

In [8]:
from pyspark.ml.feature import StringIndexer

In [9]:
indexer = StringIndexer(inputCol="category", outputCol="category_indexed" )
df_r = indexer.fit(df).transform(df)
df_r.show() 

+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+----------------+
|squareMeters|numberOfRooms|hasYard|hasPool|floors|cityCode|cityPartRange|numPrevOwners|made|isNewBuilt|hasStormProtector|basement|attic|garage|hasStorageRoom|hasGuestRoom|    price|category|category_indexed|
+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+----------------+
|       75523|            3|      0|      1|    63|    9373|            3|            8|2005|         0|                1|    4313| 9005|   956|             0|           7|7559081.5|   Basic|             0.0|
|       80771|           39|      1|      1|    98|   39381|            8|            6|2015|         1|                0|    3653| 2436|   128|             1|     

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
df.groupBy("cityCode").count().sort("count", ascending = False).show(25)    # sroting by groupcodes

+--------+-----+
|cityCode|count|
+--------+-----+
|   16401|    3|
|   95054|    3|
|   98233|    3|
|   82521|    3|
|   39297|    3|
|   97260|    3|
|   37363|    3|
|   92628|    3|
|   83194|    3|
|   79444|    3|
|   56356|    3|
|   36929|    3|
|   96283|    3|
|   14423|    2|
|   52507|    2|
|    8350|    2|
|   16503|    2|
|   66810|    2|
|   56384|    2|
|   49748|    2|
|   46266|    2|
|   31987|    2|
|    5308|    2|
|   90218|    2|
|   69329|    2|
+--------+-----+
only showing top 25 rows



In [13]:
#train_data,test_data=df_r.randomSplit([.8,.2])
#test_data.show(truncate=False,n=3)

In [12]:
#coef_var= ['squareMeters', 'numberOfRooms', 'cityCode', 'isNewBuilt', 'category_indexed']
vecAssembler = VectorAssembler(inputCols = ['squareMeters', 'numberOfRooms', 'cityCode', 'isNewBuilt', 'category_indexed'], outputCol = 'features')
output=vecAssembler.transform(df_r)
output.show(truncate=False,n=5)

+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+----------------+------------------------------+
|squareMeters|numberOfRooms|hasYard|hasPool|floors|cityCode|cityPartRange|numPrevOwners|made|isNewBuilt|hasStormProtector|basement|attic|garage|hasStorageRoom|hasGuestRoom|price    |category|category_indexed|features                      |
+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+----------------+------------------------------+
|75523       |3            |0      |1      |63    |9373    |3            |8            |2005|0         |1                |4313    |9005 |956   |0             |7           |7559081.5|Basic   |0.0             |[75523.0,3.0,9373.0,0.0,0.0]  |
|80771       |39           |1      |1   

In [15]:
#output= vecAssembler.transform(df_r)

In [13]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[75523.0,3.0,9373...|
|[80771.0,39.0,393...|
|[55712.0,58.0,344...|
|[32316.0,47.0,279...|
|[70429.0,19.0,380...|
|[39223.0,36.0,394...|
|[58682.0,10.0,645...|
|[86929.0,100.0,98...|
|[51522.0,3.0,9047...|
|[39686.0,42.0,710...|
|[23563.0,21.0,910...|
|[96470.0,74.0,920...|
|[19127.0,31.0,747...|
|[13087.0,44.0,404...|
|[79770.0,3.0,5481...|
|[75985.0,60.0,651...|
|[64169.0,88.0,617...|
|[99371.0,31.0,962...|
|[25966.0,37.0,228...|
|[41792.0,43.0,807...|
+--------------------+
only showing top 20 rows



In [14]:
output.show()

+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+----------------+--------------------+
|squareMeters|numberOfRooms|hasYard|hasPool|floors|cityCode|cityPartRange|numPrevOwners|made|isNewBuilt|hasStormProtector|basement|attic|garage|hasStorageRoom|hasGuestRoom|    price|category|category_indexed|            features|
+------------+-------------+-------+-------+------+--------+-------------+-------------+----+----------+-----------------+--------+-----+------+--------------+------------+---------+--------+----------------+--------------------+
|       75523|            3|      0|      1|    63|    9373|            3|            8|2005|         0|                1|    4313| 9005|   956|             0|           7|7559081.5|   Basic|             0.0|[75523.0,3.0,9373...|
|       80771|           39|      1|      1|    98|   39381|            8|      

In [17]:
#final_data = output.select("features", "price").show()
#data1=output.na.drop()
final_data = output.select("features", "price")

In [15]:
import pandas as pd
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplitModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
import tempfile 
from pyspark.sql.functions import pandas_udf


In [16]:
final_data = output.select("features", "price")
final_data.show()

+--------------------+---------+
|            features|    price|
+--------------------+---------+
|[75523.0,3.0,9373...|7559081.5|
|[80771.0,39.0,393...|8085989.5|
|[55712.0,58.0,344...|5574642.1|
|[32316.0,47.0,279...|3232561.2|
|[70429.0,19.0,380...|7055052.0|
|[39223.0,36.0,394...|3926647.2|
|[58682.0,10.0,645...|5876376.5|
|[86929.0,100.0,98...|8696869.3|
|[51522.0,3.0,9047...|5154055.2|
|[39686.0,42.0,710...|3970892.1|
|[23563.0,21.0,910...|2366397.3|
|[96470.0,74.0,920...|9652258.1|
|[19127.0,31.0,747...|1914688.8|
|[13087.0,44.0,404...|1320803.4|
|[79770.0,3.0,5481...|7986665.8|
|[75985.0,60.0,651...|7607322.9|
|[64169.0,88.0,617...|6420823.1|
|[99371.0,31.0,962...|9944705.3|
|[25966.0,37.0,228...|2604486.6|
|[41792.0,43.0,807...|4187667.7|
+--------------------+---------+
only showing top 20 rows



In [17]:
train_data,test_data=final_data.randomSplit([.8,.2])
train_data.show()
test_data.show()

+--------------------+-------+
|            features|  price|
+--------------------+-------+
|[89.0,33.0,61648....|10313.5|
|[111.0,32.0,80623...|14431.3|
|[123.0,61.0,33633...|13229.1|
|[137.0,32.0,40384...|21201.1|
|[141.0,16.0,42248...|22670.7|
|[143.0,27.0,82480...|17071.0|
|[149.0,28.0,63045...|23653.1|
|[152.0,80.0,9849....|22160.2|
|[153.0,71.0,99905...|22499.2|
|[163.0,35.0,29281...|19788.5|
|[180.0,63.0,91418...|19638.0|
|[187.0,96.0,10784...|26533.9|
|[196.0,56.0,53558...|23818.4|
|[202.0,18.0,38322...|28295.6|
|[210.0,75.0,25150...|28028.0|
|[211.0,10.0,40481...|24058.9|
|[229.0,49.0,72140...|27438.4|
|[235.0,86.0,42101...|28656.5|
|[241.0,78.0,86182...|29643.3|
|[255.0,62.0,68676...|34291.5|
+--------------------+-------+
only showing top 20 rows

+--------------------+-------+
|            features|  price|
+--------------------+-------+
|[128.0,38.0,8862....|15488.0|
|[148.0,91.0,27008...|16799.2|
|[163.0,6.0,26761....|17363.0|
|[176.0,30.0,62932...|18985.0|
|[302.0,23.0,

In [18]:
from pyspark.ml.regression import LinearRegression as LR

regressor=LR(featuresCol='features', labelCol='price')
model=regressor.fit(train_data.select(["features","price"]))




In [19]:
model.coefficients

DenseVector([99.9994, 2.3592, -0.0001, -801.4571, 3960.6618])

In [20]:
model.intercept

6262.738848455128

In [45]:
model.summary

In [50]:
import statsmodels.api as sm


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [53]:
print(model.summary)

In [33]:
#predictions
pred_results = model.evaluate(test_data)
pred_results.predictions.show(truncate=False)


+----------------------------+-------+------------------+
|features                    |price  |prediction        |
+----------------------------+-------+------------------+
|[128.0,38.0,8862.0,0.0,0.0] |15488.0|19151.444394847516|
|[148.0,91.0,27008.0,1.0,0.0]|16799.2|20473.24051303907 |
|[163.0,6.0,26761.0,1.0,0.0] |17363.0|21772.72211083305 |
|[176.0,30.0,62932.0,0.0,0.0]|18985.0|23927.258286932512|
|[302.0,23.0,81623.0,1.0,0.0]|34373.4|35707.382993141844|
|[303.0,75.0,7493.0,1.0,0.0] |31966.1|35937.30435208021 |
|[371.0,29.0,34470.0,1.0,0.0]|45524.8|42626.10283894738 |
|[405.0,38.0,19931.0,0.0,0.0]|54382.3|46850.1924157794  |
|[416.0,46.0,62199.0,0.0,0.0]|46790.6|47964.929626363686|
|[429.0,58.0,40696.0,0.0,0.0]|53940.1|49295.333131953514|
|[464.0,16.0,84350.0,1.0,0.0]|52352.6|51890.5023350832  |
|[600.0,37.0,72736.0,1.0,0.0]|63402.1|65541.09687854779 |
|[617.0,21.0,44204.0,0.0,0.0]|67453.3|68007.5837043433  |
|[634.0,64.0,40942.0,1.0,0.0]|66144.0|69007.88115735695 |
|[636.0,47.0,2

In [ ]:
### PErformance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

(0.9999989362777192, 2386.259923939457, 8943752.020623468)

In [25]:
from pyspark.sql.functions import corr


In [29]:
df.select(corr('squareMeters', 'price')).show()

+-------------------------+
|corr(squareMeters, price)|
+-------------------------+
|       0.9999993570640745|
+-------------------------+



In [30]:
df.select(corr('citycode', 'price')).show()

+---------------------+
|corr(citycode, price)|
+---------------------+
| -0.00153936734858...|
+---------------------+



In [31]:
df.select(corr('numberofRooms', 'price')).show()

+--------------------------+
|corr(numberofRooms, price)|
+--------------------------+
|      0.009590905935479132|
+--------------------------+



In [32]:
df.select(corr('isNewBuilt', 'price')).show()

+-----------------------+
|corr(isNewBuilt, price)|
+-----------------------+
|    -0.0106427743595189|
+-----------------------+



# New Section